#### Ejercicio 4 :
- [x] Leer tabla de base de datos postgreSql mediante jdbe
- [x] Escribir tabla en base de datos postgreSql mediante jdbc

In [74]:
%run init_spark.py

In [75]:
jdbc_url = "jdbc:postgresql://localhost:5432/sparkdb"

connection_properties = {
    "user": "sparkuser",
    "password": "sparkpassword",
    "driver": "org.postgresql.Driver"
}

In [76]:
try:
    df = spark.read.jdbc(url=jdbc_url, table="people", properties=connection_properties)
except Exception as e:
    print(f"Error: {e}")

# Ordenar el DataFrame por la columna 'id'
df_ordered = df.orderBy("id")
df_ordered.show()

+---+-------+---+------+
| id|   name|age|gender|
+---+-------+---+------+
|  1|Charlie| 28|     M|
|  2|  Diana| 22|     F|
+---+-------+---+------+



In [77]:
new_data = [("Charlie", 28, "M"), ("Diana", 22, "F")]
columns = ["name", "age", "gender"]

new_df = spark.createDataFrame(new_data, columns)

In [78]:
try:
    new_df.write.jdbc(url=jdbc_url, table="people", mode="append", properties=connection_properties)
    print("Data written successfully.")
except Exception as e:
    print(f"Error: {e}")

Data written successfully.


In [79]:
try:
    df = spark.read.jdbc(url=jdbc_url, table="people", properties=connection_properties)
except Exception as e:
    print(f"Error: {e}")

df_ordered = df.orderBy("id")
df_ordered.show()

+---+-------+---+------+
| id|   name|age|gender|
+---+-------+---+------+
|  1|Charlie| 28|     M|
|  2|  Diana| 22|     F|
|  3|Charlie| 28|     M|
|  4|  Diana| 22|     F|
+---+-------+---+------+



In [80]:
# %pip install jaydebeapi
import jaydebeapi

In [81]:
try:
    conn = jaydebeapi.connect(
        connection_properties["driver"],
        jdbc_url,
        [connection_properties["user"], connection_properties["password"]],
        jdbc_jar_path
    )
    conn.jconn.setAutoCommit(False)  # Deshabilitar auto-commit
    curs = conn.cursor()
    
    # Eliminar todos los registros de la tabla 'people'
    delete_query = "DELETE FROM people"
    curs.execute(delete_query)
    
    # Reiniciar la secuencia de 'id' a 1
    reset_sequence_query = "ALTER SEQUENCE people_id_seq RESTART WITH 1"
    curs.execute(reset_sequence_query)
    
    conn.commit()
    curs.close()
    conn.close()
    print("Todos los registros de la tabla 'people' han sido eliminados y la secuencia ha sido reiniciada.")
except Exception as e:
    print(f"Error: {e}")

Todos los registros de la tabla 'people' han sido eliminados y la secuencia ha sido reiniciada.
